# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,557.884339,-0.352558,1234,557.531780,558.236897,1115.063560,1116.473794
6,817.716039,-0.430684,1234,817.285356,818.146723,1634.570711,1636.293447
7,854.502741,-0.465794,1234,854.036947,854.968535,1708.073894,1709.937071
8,906.522862,0.524700,1234,907.047562,905.998162,1814.095124,1811.996325
9,759.938278,0.699906,1234,760.638184,759.238372,1521.276369,1518.476744
10,917.752331,0.935084,1234,918.687414,916.817247,1837.374828,1833.634494
11,772.287803,0.148241,1234,772.436045,772.139562,1544.872089,1544.279123
12,113.659515,-0.786855,1234,112.872660,114.446369,225.745320,228.892739
13,43.845183,-0.315256,1234,43.529927,44.160440,87.059853,88.320879
14,446.197283,0.181836,1234,446.379119,446.015447,892.758237,892.030895


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-557.531780,558.236897
6,-817.285356,818.146723
7,-854.036947,854.968535
8,-907.047562,905.998162
9,-760.638184,759.238372
10,-918.687414,916.817247
11,-772.436045,772.139562
12,-112.872660,114.446369
13,-43.529927,44.160440
14,-446.379119,446.015447


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-557.531780,558.236897
6,-817.285356,818.146723
7,-854.036947,854.968535
8,-907.047562,905.998162
9,-760.638184,759.238372
10,-918.687414,916.817247
11,-772.436045,772.139562
12,-112.872660,114.446369
13,-43.529927,44.160440
14,-446.379119,446.015447


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-557.53178012,  558.23689701],
       [-817.28535557,  818.14672341],
       [-854.0369471 ,  854.96853532],
       [-907.04756203,  905.99816235],
       [-760.63818447,  759.2383718 ],
       [-918.68741411,  916.81724703],
       [-772.43604465,  772.1395617 ],
       [-112.87266019,  114.44636948],
       [ -43.52992667,   44.16043954],
       [-446.3791187 ,  446.01544727]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1113.063560,-1116.473794
6,1632.570711,-1636.293447
7,1706.073894,-1709.937071
8,1812.095124,-1811.996325
9,1519.276369,-1518.476744
10,1835.374828,-1833.634494
11,1542.872089,-1544.279123
12,223.745320,-228.892739
13,85.059853,-88.320879
14,890.758237,-892.030895


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.557884,0.323721
6,0.817716,0.284658
7,0.854503,0.267103
8,0.906523,0.762350
9,0.759938,0.849953
10,0.917752,0.967542
11,0.772288,0.574121
12,0.113660,0.106573
13,0.043845,0.342372
14,0.446197,0.590918


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,557.884339,-0.352558
6,817.716039,-0.430684
7,854.502741,-0.465794
8,906.522862,0.524700
9,759.938278,0.699906
10,917.752331,0.935084
11,772.287803,0.148241
12,113.659515,-0.786855
13,43.845183,-0.315256
14,446.197283,0.181836


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
